In [1]:
import pandas as pd
import numpy as np
import re

import json

from IPython.display import JSON

In [62]:
data = pd.read_csv('chicago-food-inspections/food-inspections.csv')

#### Preprocess the data

In [92]:
establishments = data[['DBA Name', 'AKA Name', 'Address', 'City', 'State']].copy().drop_duplicates().reset_index()
establishments.isnull().any()

index       False
DBA Name    False
AKA Name     True
Address     False
City         True
State        True
dtype: bool

In [93]:
# Add two columns to relate to Yelp data
establishments['id'] = None
establishments['matches'] = -1

establishments.loc[establishments['City'].isnull(), 'matches'] = -2
establishments.loc[establishments['City'].isnull(), 'matches'] = -2

In [97]:
establishments.nunique()

index       32373
DBA Name    27476
AKA Name    26210
Address     18035
City           71
State           4
id              0
matches         2
dtype: int64

## Yelp Database

In [2]:
import os
from yelpapi import YelpAPI

YELP_API_KEY = os.environ['YELP_API_KEY']
YELP_CLIENT_ID = os.environ['YELP_CLIENT_ID']
client = YelpAPI(YELP_API_KEY)

#### Read already acquired data

In [3]:
# The yelp_df is a selection of columns of the original df, containing only location information
# We then drop all duplicates and reset the index in order to more easily cross reference with the new data
# We also include a id col corresponding to the yelp id, and a count of how many results returned
yelp_df = pd.read_pickle('yelp/yelp.pkl')

# Contains the scrapped data from yelp
scrapped = pd.read_pickle('yelp/scrapped.pkl')

# Contains remaining data such as reviews, review count, price, and food categories
business = pd.read_pickle('yelp/business.pkl')

# For each business, we get 3 reviews with the start of the text and a rating 
reviews = pd.read_pickle('yelp/reviews.pkl')

### Functions to get data from Yelp

#### Match establishments to Yelp and get ID along with address information

In [4]:
def business_match_to_df_row(business_):
    business = business_.copy()
    business.update(business.pop('coordinates'))
    business.update(business.pop('location'))
    business.pop('phone')
    business.pop('display_phone')
    business.pop('country')
    if str(business['address2']) == '' or not business['address2']:
        del business['address2']
    if str(business['address3']) == '' or not business['address3']:
        business.pop('address3', None)
    return business

In [148]:
def match_rows_to_yelp(min_range, max_range,yelp_df,scrapped):
    for i in range(min_range,max_range):
        # Get the i-th row of the data frame
        row = yelp_df.iloc[i]
        
        # Special case when our search terms contain NaN
        if row['matches'] == -2:
            continue
            
        # We haven't requested the data yet and no NaN im the request
        if row['matches'] == -1:
            # Some rows have only DBA / AKA
            row_name = row['DBA Name'] if row[['AKA Name']].isnull().values.any() else row['AKA Name']
            
            query = client.business_match_query(name=row_name,
                                                city=row['City'],
                                                state=row['State'],
                                                country='US',
                                                address1=row['Address'])
            
            # We note how many matches we got and only take the first one
            yelp_df.loc[yelp_df.index[i], 'matches'] = len(query['businesses'])
            
            if len(query['businesses']) == 1:
                yelp_df.loc[yelp_df.index[i], 'id'] = query['businesses'][0]['id']
                scrapped = scrapped.append(pd.DataFrame([business_match_to_df_row(query['businesses'][0])]))
            continue
              
        # We have more matches and need to examine more closely which one to insert
        if row['matches'] == 2:
            continue
                
match_rows_to_yelp(0,10000,yelp_df,scrapped)

/usr/local/anaconda3/envs/ada/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


#### Get more information for each business

In [108]:
def get_business_details(min_range, max_range, scrapped, business):
    assert(min_range <= max_range and 0 <= min_range and max_range<len(scrapped.index))
    
    # Get all other information from business
    wanted_business = ['id','is_closed','review_count','categories','rating','price','transactions']
    for i in range(min_range,max_range):
        row = scrapped.iloc[i]
        if not business['id'].isin([row['id']]).any():
            query = client.business_query(id=row['id'])
            result = dict((k, query[k]) for k in wanted_business if k in query)
            business = business.append([result])
                
get_business_details(0,10,scrapped,business)

#### Get reviews

In [111]:
def get_reviews(min_range, max_range, scrapped, reviews):
    assert(min_range <= max_range and 0 <= min_range and max_range<len(scrapped.index))
    
    wanted_reviews = ['id','text','rating','time_created']
    for i in range(min_range,max_range):
        row = scrapped.iloc[i]
        if not reviews['id'].isin([row['id']]).any():
            query = client.reviews_query(id=row['id'])
            for review in query['reviews']:
                result = dict((k, review[k]) for k in wanted_reviews if k in review)
                result.update({'business_id':row['id']})
                reviews = reviews.append([result])
get_reviews(0, 10, scrapped, reviews)

## Various lines to combine the three data frames

In [104]:
# Compare the original data with scraped and combine with more business details
yelp_df[yelp_df['id'].notnull()].merge(scrapped,on='id').merge(business,on='id')

,index,DBA Name,AKA Name,Address,City,State,id,matches,address1,address2,...,longitude,name,state,zip_code,is_closed,review_count,categories,rating,price,transactions
0,5,CRAZY BIRD,CRAZY BIRD,1160 W GRAND AVE,CHICAGO,IL,7xiYs3CWFbs9lnqor7iRpQ,1,1160 W Grand Ave,NaN,...,-87.656985,Crazy Bird,IL,60642,False,75,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.5,$,"[pickup, delivery]"
1,6,THE REDHEAD PIANO BAR,THE REDHEAD PIANO BAR,16-18 W ONTARIO ST,CHICAGO,IL,KPblL_UbpgEwUbZEUscTvA,1,16 W Ontario St,NaN,...,-87.628740,Redhead Piano Bar,IL,60654,False,529,"[{'alias': 'musicvenues', 'title': 'Music Venu...",3.0,$$,[]
2,7,HAISOUS,CA PHE DA,1800-1802 S CARPENTER ST,CHICAGO,IL,GkW4K_g-wtNCHuF0w8LMYA,1,1800 1/2 S Carpenter St,NaN,...,-87.653500,Ca Phe Da Vietnamese Cafe,IL,60608,False,144,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,$$,"[pickup, delivery]"
3,8,HARMONY RESTAURANT,HARMONY RESTAURANT,6525 W ARCHER AVE,CHICAGO,IL,yz-3hfOJYKRnPRBOFPpHqQ,1,6525 W Archer Ave,NaN,...,-87.785297,Harmony Restaurant,IL,60638,False,129,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.0,$$,"[pickup, delivery]"
4,10,HIMALAYAN SHERPA KITCHEN,HIMALAYAN SHERPA KITCHEN,2701 W LAWRENCE AVE,CHICAGO,IL,2D4YeUTn2GGtrcLoHSpvJQ,1,2701 W Lawrence Ave,NaN,...,-87.696393,Himalayan Sherpa Kitchen,IL,60625,False,23,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",4.5,NaN,"[delivery, pickup]"
5,11,7 ELEVEN #35750H,7 ELEVEN,1658 N MILWAUKEE AVE,CHICAGO,IL,Q-0hZSru1U4xSXoM6MdjtA,1,1658 N Milwaukee,NaN,...,-87.679586,7-Eleven,IL,60647,False,5,"[{'alias': 'convenience', 'title': 'Convenienc...",3.0,$,[]
6,11,7 ELEVEN #35750H,7 ELEVEN,1658 N MILWAUKEE AVE,CHICAGO,IL,Q-0hZSru1U4xSXoM6MdjtA,1,1658 N Milwaukee,NaN,...,-87.679586,7-Eleven,IL,60647,False,5,"[{'alias': 'convenience', 'title': 'Convenienc...",3.0,$,[]
7,13,7-ELEVEN #35750A,7-ELEVEN,1658 N MILWAUKEE AVE,CHICAGO,IL,Q-0hZSru1U4xSXoM6MdjtA,1,1658 N Milwaukee,NaN,...,-87.679586,7-Eleven,IL,60647,False,5,"[{'alias': 'convenience', 'title': 'Convenienc...",3.0,$,[]
8,13,7-ELEVEN #35750A,7-ELEVEN,1658 N MILWAUKEE AVE,CHICAGO,IL,Q-0hZSru1U4xSXoM6MdjtA,1,1658 N Milwaukee,NaN,...,-87.679586,7-Eleven,IL,60647,False,5,"[{'alias': 'convenience', 'title': 'Convenienc...",3.0,$,[]
9,12,WALGREENS #10771,WALGREENS #10771,6460 W FULLERTON AVE,CHICAGO,IL,Qk7K6Y1S6rmduUNch70bsA,1,6460 W Fullerton Ave,NaN,...,-87.788128,Walgreens,IL,60707,True,4,"[{'alias': 'drugstores', 'title': 'Drugstores'...",2.5,$$,[]


In [112]:
df = yelp_df[yelp_df['id'].notnull()].merge(scrapped,on='id')

In [125]:
yelp_df

,index,DBA Name,AKA Name,Address,City,State,id,matches
0,0,SERENDIPITY CHILDCARE,SERENDIPITY CHILDCARE,1300 W 99TH ST,CHICAGO,IL,sPBHUHyv7fwgcPs4jy5LMg,1
1,1,YOLK TEST KITCHEN,YOLK TEST KITCHEN,1767 N MILWAUKEE AVE,CHICAGO,IL,heSNmi80snMFaD2aqux8EA,1
2,2,LAS ASADAS MEXICAN GRILL,LAS ASADAS MEXICAN GRILL,3834 W 47TH ST,CHICAGO,IL,btDjEovu7jypI9vCRe7q8A,1
3,3,LA PALAPITA,LA PALAPITA,3834 W 47TH ST,CHICAGO,IL,None,0
4,4,47TH ST CANTINA,47TH ST CANTINA,4311 W 47TH ST,CHICAGO,IL,None,0
...,...,...,...,...,...,...,...,...
32369,194730,"SURF & TURF WHERE SEAFOOD & STEAK MEET, INC.","SURF & TURF WHERE SEAFOOD & STEAK MEET, INC.",13205 S MUSKEGON AVE,CHICAGO,IL,None,-1
32370,194746,SAFAH FOOD & LIQUOR INC,SAFAH FOOD & LIQUOR INC,7105 S RACINE AVE,CHICAGO,IL,None,-1
32371,194758,MAKIA FOOD,MAKIA FOOD,2458 S CALIFORNIA AVE,CHICAGO,IL,None,-1
32372,194763,DUNKIN DONUTS,NaN,970 W PERSHING RD,CHICAGO,IL,None,-1


In [126]:
scrapped

,address1,address2,address3,alias,city,display_address,id,latitude,longitude,name,state,zip_code
0,1160 W Grand Ave,NaN,NaN,crazy-bird-chicago-2,Chicago,"[1160 W Grand Ave, Chicago, IL 60642]",7xiYs3CWFbs9lnqor7iRpQ,41.891326,-87.656985,Crazy Bird,IL,60642
0,16 W Ontario St,NaN,NaN,the-redhead-piano-bar-chicago,Chicago,"[16 W Ontario St, Chicago, IL 60654]",KPblL_UbpgEwUbZEUscTvA,41.893530,-87.628740,Redhead Piano Bar,IL,60654
0,1800 1/2 S Carpenter St,NaN,NaN,ca-phe-da-vietnamese-cafe-chicago,Chicago,"[1800 1/2 S Carpenter St, Chicago, IL 60608]",GkW4K_g-wtNCHuF0w8LMYA,41.857870,-87.653500,Ca Phe Da Vietnamese Cafe,IL,60608
0,6525 W Archer Ave,NaN,NaN,harmony-restaurant-chicago,Chicago,"[6525 W Archer Ave, Chicago, IL 60638]",yz-3hfOJYKRnPRBOFPpHqQ,41.792210,-87.785297,Harmony Restaurant,IL,60638
0,2701 W Lawrence Ave,NaN,NaN,himalayan-sherpa-kitchen-chicago,Chicago,"[2701 W Lawrence Ave, Chicago, IL 60625]",2D4YeUTn2GGtrcLoHSpvJQ,41.968459,-87.696393,Himalayan Sherpa Kitchen,IL,60625
...,...,...,...,...,...,...,...,...,...,...,...,...
0,215 W Jackson Blvd,NaN,NaN,lukes-italian-beef-chicago,Chicago,"[215 W Jackson Blvd, Chicago, IL 60606]",vqCfdZCRH7h6eAB0U7ANGg,41.877969,-87.634405,Luke's Italian Beef,IL,60606
0,1659 W Ogden Ave,NaN,NaN,the-ogden-chicago-4,Chicago,"[1659 W Ogden Ave, Chicago, IL 60612]",ecj6b2CGcThT_1tQSxnStA,41.879758,-87.667800,The Ogden,IL,60612
0,3002 N Sheffield Ave,NaN,NaN,panes-bread-cafe-chicago,Chicago,"[3002 N Sheffield Ave, Chicago, IL 60657]",skpDldmlZlw6e82tpiVyrg,41.936486,-87.654205,Panes Bread Cafe,IL,60657
0,3104 N Central Ave,NaN,NaN,tasty-polish-food-chicago,Chicago,"[3104 N Central Ave, Chicago, IL 60634]",fcn5oDHf_3Ml8_eTgjcbJw,41.937031,-87.766792,Tasty Polish Food,IL,60634


In [127]:
business

,id,is_closed,review_count,categories,rating,price,transactions
0,7xiYs3CWFbs9lnqor7iRpQ,False,75,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.5,$,"[pickup, delivery]"
0,KPblL_UbpgEwUbZEUscTvA,False,529,"[{'alias': 'musicvenues', 'title': 'Music Venu...",3.0,$$,[]
0,GkW4K_g-wtNCHuF0w8LMYA,False,144,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,$$,"[pickup, delivery]"
0,yz-3hfOJYKRnPRBOFPpHqQ,False,129,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.0,$$,"[pickup, delivery]"
0,2D4YeUTn2GGtrcLoHSpvJQ,False,23,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",4.5,NaN,"[delivery, pickup]"
0,Q-0hZSru1U4xSXoM6MdjtA,False,5,"[{'alias': 'convenience', 'title': 'Convenienc...",3.0,$,[]
0,Qk7K6Y1S6rmduUNch70bsA,True,4,"[{'alias': 'drugstores', 'title': 'Drugstores'...",2.5,$$,[]
0,dv8o6WDhQB9SkQiS0kytrQ,False,2,"[{'alias': 'tradamerican', 'title': 'American ...",3.0,$,[]
0,TZv2SNMJ7jnF1dcre3bLPw,False,4,"[{'alias': 'pizza', 'title': 'Pizza'}]",1.5,$,"[pickup, delivery]"


In [139]:
# Get number of matched businesses

len(yelp_df[yelp_df['matches'] > 0])/len(yelp_df[yelp_df['matches'] >= 0])

0.6938117986890345

In [132]:
reviews

,id,text,rating,time_created,business_id
0,Ybp-1fP53VzilpbwYkYVPA,Outstanding service and a great meal! \nI was ...,5,2019-11-07 18:09:09,7xiYs3CWFbs9lnqor7iRpQ
0,FqrJ2anEaVI05ZYoPPAY2w,"I really enjoyed this place! I split a ""whole ...",4,2019-10-25 09:22:26,7xiYs3CWFbs9lnqor7iRpQ
0,7ZDHGmevUF__oUU3b_O9Zw,Awesome place. We stopped in for a bite and go...,5,2019-08-24 17:28:05,7xiYs3CWFbs9lnqor7iRpQ
0,Ybp-1fP53VzilpbwYkYVPA,Outstanding service and a great meal! \nI was ...,5,2019-11-07 18:09:09,7xiYs3CWFbs9lnqor7iRpQ
0,FqrJ2anEaVI05ZYoPPAY2w,"I really enjoyed this place! I split a ""whole ...",4,2019-10-25 09:22:26,7xiYs3CWFbs9lnqor7iRpQ
0,7ZDHGmevUF__oUU3b_O9Zw,Awesome place. We stopped in for a bite and go...,5,2019-08-24 17:28:05,7xiYs3CWFbs9lnqor7iRpQ
0,Ybp-1fP53VzilpbwYkYVPA,Outstanding service and a great meal! \nI was ...,5,2019-11-07 18:09:09,KPblL_UbpgEwUbZEUscTvA
0,FqrJ2anEaVI05ZYoPPAY2w,"I really enjoyed this place! I split a ""whole ...",4,2019-10-25 09:22:26,KPblL_UbpgEwUbZEUscTvA
0,7ZDHGmevUF__oUU3b_O9Zw,Awesome place. We stopped in for a bite and go...,5,2019-08-24 17:28:05,KPblL_UbpgEwUbZEUscTvA
0,xkBX8B7IvDPYb_pEjyuGpQ,I just have to say Kaleen Dolan is the reason ...,5,2019-10-26 15:58:27,GkW4K_g-wtNCHuF0w8LMYA


## Save the all data

In [112]:
yelp_df.to_pickle('yelp/yelp.pkl')
scrapped.to_pickle('yelp/scrapped.pkl')
business.to_pickle('yelp/business.pkl')
reviews.to_pickle('yelp/reviews.pkl')